# Feature Engineering → Define enrichment parameters (team size, leadership, budget), test their extraction
Define - which parameters to extracts Sales and Sven (discuss feedback, send a another survey)
Test - with (our and external) NLPs and LLMs
For some params - agent should also have a search capacity
(external tools) OpenAI (DeepSeek) - ?/10 quality
(our own model) BERT
BERT - ?/10 quality
other...
Iterate (check if it works, get approved by Sven and scales)
Check which params work and do not work

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_cv_sample = pd.read_excel('30_sample_users_cleaned.xlsx')
enrichment_params = pd.read_excel('ACP - Enrichment Parameters.xlsx')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
df_cv_sample.head()

,profile_id,location,country,languages,cv_skills,industry_skills,function_skills,tags,education,experience,salary,day_rate,open_for_perm,open_for_project
0,1644,M√ºnchen,GERMANY,NaN,"[""analytik"", ""schwerpunkt"", ""onboarding"", ""wer...","[""WHOLESALE"", ""INSURANCE"", ""CONSUMER_GOODS"", ""...","[""ECOMMERCE_AND_DIGITAL_MARKETING"", ""CUSTOMER_...","[""strategisches Denken"", ""freelancer"", ""leader...","[{""title"" : ""Diplom-Soziologe"", ""organization_...","[{""title"" : ""Selbstst√§ndige Beraterin & Proje...",FROM_100K_TO_150K,BELOW_1000,True,True
1,1641,W√ºlfrath,GERMANY,NaN,"[""management"", ""booking (resource planning sof...","[""PLANT_AND_MACHINERY"", ""CONSTRUCTION_INDUSTRY...","[""GROWTH_TREND_ANALYSIS"", ""RISK_MANAGEMENT"", ""...","[""CashFlow Management "", ""Change Management"", ...",NaN,"[{""title"" : ""Head of Finance"", ""organization_n...",NONE,NONE,True,True
2,1637,M√ºlheim an der Ruhr,GERMANY,NaN,"[""facility management"", ""projektleistung"", ""be...","[""FACILITY_SERVICES_AND_SECURITY""]","[""M_AND_A_SUPPORT"", ""MARKET_ENTRY_STRATEGY"", ""...","[""fuehrung"", ""Business Development""]","[{""title"" : ""Betriebswirt (VWA),"", ""organizati...","[{""title"" : ""Gesch√§ftsf√ºhrer"", ""organization...",FROM_150K_TO_200K,FROM_1000_TO_1200,True,True
3,1633,Viersen,GERMANY,NaN,"[""buchhaltung"", ""planung"", ""verwaltung"", ""verh...","[""RETAIL_INDUSTRY"", ""FOOD_AND_BEVERAGE_INDUSTR...","[""M_AND_A_SUPPORT"", ""CHANGE_MANAGEMENT"", ""REST...","[""Team"", ""Powerpoint"", ""Excel"", ""freelancer"", ...","[{""title"" : ""Deutsches und Europ√§isches Wirts...","[{""title"" : ""Gesch√§ftsf√ºhrer"", ""organization...",FROM_200K_TO_250K,FROM_1500_TO_2000,True,True
4,1632,M√ºnchen,GERMANY,NaN,"[""account management"", ""swimming"", ""personal c...","[""PROFESSIONAL_SERVICE_FIRMS""]","[""GROWTH_TREND_ANALYSIS"", ""BENCHMARKING_AND_CO...","[""communication"", ""networking"", ""business buil...","[{""title"" : ""Doctorate (Dr. rer. oec./PhD) in ...",NaN,NONE,NONE,True,True


In [4]:
# Check type of data in each column
df_cv_sample.dtypes

profile_id           int64
location            object
country             object
languages           object
cv_skills           object
industry_skills     object
function_skills     object
tags                object
education           object
experience          object
salary              object
day_rate            object
open_for_perm         bool
open_for_project      bool
dtype: object

In [5]:
# Check for missing values
df_cv_sample.isnull().sum()

# Fill missing values with 0
df_cv_sample.fillna(0, inplace=True)

In [6]:
# Print column names
df_cv_sample.columns

Index(['profile_id', 'location', 'country', 'languages', 'cv_skills',
       'industry_skills', 'function_skills', 'tags', 'education', 'experience',
       'salary', 'day_rate', 'open_for_perm', 'open_for_project'],
      dtype='object')

In [7]:
df_cv_sample.experience[0]

'[{"title" : "Selbstst√§ndige Beraterin & Projektmanagement ", "organization_name" : "SLEEVES UP CONSULTANCY", "start_date" : "2016-05-01", "end_date" : null, "city" : "M√ºnchen"}, {"title" : "Head of Marketing Child Safety", "organization_name" : "CYBEX GMBH", "start_date" : "2020-12-01", "end_date" : "2022-12-01", "city" : "Bayreuth"}, {"title" : "Global Business Director", "organization_name" : "SPARK44 LTD", "start_date" : "2014-11-01", "end_date" : "2016-04-01", "city" : "London"}, {"title" : "Client Service Director", "organization_name" : "K√ñCKRITZD√ñRRICH GMBH", "start_date" : "2014-11-01", "end_date" : "2016-04-01", "city" : "Reutlingen"}, {"title" : "Group Account Director", "organization_name" : "PANAMA WERBEAGENTUR", "start_date" : "2009-02-01", "end_date" : "2011-03-01", "city" : "Stuttgart"}, {"title" : "Advertising Account Manager", "organization_name" : "JUNG VON MATT", "start_date" : "2008-07-01", "end_date" : "2009-01-01", "city" : "Stuttgart"}, {"title" : "Projekt M

In [8]:
enrichment_params.head()

,Enrichment Parameter,Priority Level,Implementation Approach,Feasibility,Expected Impact,Example Data,Priority - Recruiting,Priority - Sven
0,Employment Pattern & Progression,High,"Compute job tenure, career growth trends via s...",Easy,Improves understanding of job stability and ca...,"Candidate worked at 4 companies over 12 years,...",Not chosen,Not chosen
1,Company Type & Sector,High,"Use external databases (Crunchbase, LinkedIn) ...",Moderate,Enhances context on candidate's industry exper...,Candidate's company: 'TechCorp Inc.' ‚ Classif...,Not chosen,Not chosen
2,Education Quality & Ranking,Medium,Cross-check university against global rankings,Easy,Adds prestige factor for screening academic ba...,Candidate studied at 'MIT' ‚ Ranked #1 globall...,Not chosen,Not chosen
3,Skill Demand & Market Relevance,High,Compare skills against industry demand reports,Moderate,Ensures skills align with current market demand,"Candidate lists 'Python, ML, Data Analysis' ‚ ...",Not chosen,Not chosen
4,Leadership Experience,High,Extract team size and direct reports via NLP +...,Moderate,Helps assess leadership experience and respons...,Candidate led a team of 15 engineers in last r...,Not chosen,Not chosen


In [ ]:
from openai import OpenAI
import pandas as pd
import json
from tqdm import tqdm
import os

os.environ["OPENAI_API_KEY"] = "None"  # Replace with your OpenAI API key

# New style client for OpenAI v1.x
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def enrich_experience(entry): #TODO: Implement websearch to get company type, industry, etc.
    # Skip non-dict entries
    if not isinstance(entry, dict):
        return {
            "company_type": "Unknown",
            "industry": "Unknown",
            "leadership_scope": "Unknown",
            "team_size_managed": "Unknown",
            "budget_responsibility": "Unknown"
        }

    company = entry.get('organization_name', 'Unknown')
    role = entry.get('title', 'Unknown')
    city = entry.get('city', 'Unknown')
    start = entry.get('start_date', 'Unknown')
    end = entry.get('end_date', 'Unknown')

    prompt = f"""
    You are an expert recruiter. Based on the following job experience details, classify:
    1. Company Type (Startup / PE-backed / Corporate / Unknown)
    2. Industry (e.g., Retail, Automotive, Consulting, etc.)
    3. Leadership Scope (Operational / Strategic / Both / Unknown)
    4. Team Size Managed (number or Unknown)
    5. Budget Responsibility (amount or Unknown)

    Company Name: {company}
    Job Title: {role}
    City: {city}
    Dates: {start} to {end}

    Return the answers in JSON format like this:
    {{
      "company_type": "PE-backed",
      "industry": "Consumer Goods",
      "leadership_scope": "Strategic",
      "team_size_managed": "10-20",
      "budget_responsibility": "€1M-€5M"
    }}

    Use web search to find the company type and industry. If you cannot find the information, return "Unknown" for those fields.
    Return only the JSON response without any additional text.
    Do not include any explanations or additional information.
    """

    try:
        # response = client.chat.completions.create(
        #     # model ='gpt-3.5-turbo',
        #     # model = 'gpt-4o-mini-search-preview',
        #     model='gpt-4o',
        #     messages=[{"role": "user", "content": prompt}],
        #     temperature=0
        # )

        response = client.responses.create(
            # model ='gpt-3.5-turbo',
            # model = 'gpt-4o-mini-search-preview',
            model='gpt-4o',
            input=[{"role": "user", "content": prompt}],
            tools=[{"type": "web_search"}],
            temperature=0
        )

        # content = response.choices[0].message.content.strip()
        # content = response.output[0].content[0].text
        content = response.output[0]
        print(json.dumps(content, default=lambda o: o.__dict__, indent=2))

        # Example output:
        #         [
        #   {
        #     "id": "ws_67f524a8d38c819196a8ed4b1e0b471509ba25e1c733fe0a",
        #     "status": "completed",
        #     "type": "web_search_call"
        #   },
        #   {
        #     "id": "msg_67f524ab25c081919f85c994d08f160109ba25e1c733fe0a",
        #     "content": [
        #       {
        #         "annotations": [],
        #         "text": "{\n  \"company_type\": \"Unknown\",\n  \"industry\": \"Consulting\",\n  \"leadership_scope\": \"Unknown\",\n  \"team_size_managed\": \"Unknown\",\n  \"budget_responsibility\": \"Unknown\"\n} ",
        #         "type": "output_text"
        #       }
        #     ],
        #     "role": "assistant",
        #     "status": "completed",
        #     "type": "message"
        #   }
        # ]
        # Extract the JSON part from the response based on the expected structure above
        # content = content[1]['content'][0]['text']

        # Convert JSON from model
        json_result = json.loads(content)
        print(json_result)


    except Exception as e:
        print(f"Error: {e}")
        json_result = {
            "company_type": "Unknown",
            "industry": "Unknown",
            "leadership_scope": "Unknown",
            "team_size_managed": "Unknown",
            "budget_responsibility": "Unknown"
        }

    return json_result

def process_experience_column(experience_column):
    enriched_profiles = []
    for exp_json in tqdm(experience_column):
        profile_results = []
        try:
            if pd.isna(exp_json) or not isinstance(exp_json, str):
                enriched_profiles.append([])
                continue

            # Fix common JSON issues
            exp_json_cleaned = (
                exp_json.replace("√§", "ä")
                        .replace("√º", "ü")
                        .replace("√ñ", "Ö")
                        .replace("'", '"')
            )
            experiences = json.loads(exp_json_cleaned)

            # Ensure it's a list of dictionaries
            if isinstance(experiences, dict):
                experiences = [experiences]
            elif not isinstance(experiences, list):
                enriched_profiles.append([])
                continue

            for entry in experiences:
                enriched = enrich_experience(entry)
                enriched_entry = {
                    "organization_name": entry.get('organization_name', ''),
                    "role": entry.get('title', ''),
                    **enriched
                }
                profile_results.append(enriched_entry)
                print(enriched_entry)

        except Exception as e:
            print(f"Error processing profile: {e}")
            profile_results.append({"error": str(e)})

        enriched_profiles.append(profile_results)

    return enriched_profiles

# Example DataFrame
df = df_cv_sample.copy().head(-1)

# Process experiences
df['enriched_experience'] = process_experience_column(df['experience'])

# Save output
df.to_json('enriched_experience_output.json', orient='records')

  0%|          | 0/29 [00:00<?, ?it/s]

{
  "id": "ws_67f535b234b08191b5fbe367f7607c1b015681796f413b7c",
  "status": "completed",
  "type": "web_search_call"
}
Error: the JSON object must be str, bytes or bytearray, not ResponseFunctionWebSearch
{'organization_name': 'SLEEVES UP CONSULTANCY', 'role': 'Selbstständige Beraterin & Projektmanagement ', 'company_type': 'Unknown', 'industry': 'Unknown', 'leadership_scope': 'Unknown', 'team_size_managed': 'Unknown', 'budget_responsibility': 'Unknown'}


  0%|          | 0/29 [00:07<?, ?it/s]


KeyboardInterrupt: 